In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers

# Image size and batch
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# ✅ Data generators with validation split
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,   # 20% data for validation
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_data = train_datagen.flow_from_directory(
    "dataset/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data = train_datagen.flow_from_directory(
    "dataset/train",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# ✅ Step 1: Build base ResNet50 model (frozen)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # freeze entire base

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(train_data.num_classes, activation='softmax')
])

# Compile
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ✅ Phase 1: Train top layers only
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

# ✅ Step 2: Fine-tuning
base_model.trainable = True

# Keep early layers frozen, unfreeze last 30 layers
for layer in base_model.layers[:-30]:
    layer.trainable = False

# Recompile with lower LR
model.compile(optimizer=optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ✅ Phase 2: Continue training with fine-tuning
history_fine = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

# Save model
model.save("tomato_disease_resnet50_finetuned.h5")


Found 3601 images belonging to 3 classes.
Found 899 images belonging to 3 classes.
Epoch 1/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 154s 1s/step - accuracy: 0.3824 - loss: 1.1893 - val_accuracy: 0.4283 - val_loss: 1.0536
Epoch 2/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.4072 - loss: 1.1005 - val_accuracy: 0.5673 - val_loss: 1.0385
Epoch 3/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.4341 - loss: 1.0659 - val_accuracy: 0.5161 - val_loss: 1.0273
Epoch 4/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 356s 3s/step - accuracy: 0.4887 - loss: 1.0433 - val_accuracy: 0.5862 - val_loss: 1.0130
Epoch 5/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 189s 2s/step - accuracy: 0.5335 - loss: 1.0176 - val_accuracy: 0.5261 - val_loss: 1.0106
Epoch 6/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.5215 - loss: 1.0154 - val_accuracy: 0.5751 - val_loss: 0.9964
Epoch 7/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.5640 - loss: 0.9984 - val_accuracy: 0.5851 - val_loss: 0.9854
Epoch 8/10
113

In [3]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 53,276,043 (203.23 MB)

 Trainable params: 14,712,835 (56.13 MB)

 Non-trainable params: 9,137,536 (34.86 MB)

 Optimizer params: 29,425,672 (112.25 MB)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

# Load trained model
model = load_model("tomato_disease_model.h5")

# Load test image
img_path = "dataset/train/early.jpg"  # put your tomato leaf image here
img = image.load_img(img_path, target_size=(128,128))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)

# Map class indices
labels = list(train_data.class_indices.keys())
print("Predicted Class:", labels[predicted_class])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step
Predicted Class: Tomato_Late_blight
